In [1]:
#-----import packages-----#

#common python packages
import numpy as np
import string
import random
import os
import pickle
import argparse
import wget
import math
import gc
import sys
import multiprocessing as mp
import matplotlib.pyplot as plt
from datetime import datetime
from tempfile import TemporaryFile

#biological packages
import pybedtools
from pybedtools import featurefuncs
import pyBigWig

#machine learning packages
import sklearn
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import pandas as pd

import tensorflow as tf

import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten, GlobalAveragePooling2D, Multiply
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.utils import Sequence, plot_model
from keras.constraints import unit_norm
from keras import regularizers
from keras.callbacks import EarlyStopping, Callback, TensorBoard, ReduceLROnPlateau
import keras_metrics as km
from keras.models import load_model


from models.v3 import create_model
from models.custom_metrics import auroc, auprc, recall_m, precision_m, f1_m

#notify the OS about GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KERAS_BACKEND'] = 'tensorflow'

/gpfs/ysm/project/zc264/conda_envs/old_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/gpfs/ysm/project/zc264/conda_envs/old_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/gpfs/ysm/project/zc264/conda_envs/old_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/gpf

In [2]:
window_size = 4000
def bigWigAverageOverBed(x, bigwig):
    return bigwig.stats(x.chrom, x.start, x.stop, nBins=int(window_size/10))

def get_signal(input_list):
    print(input_list)
    sys.stdout.flush()
    return [bigWigAverageOverBed(x, pyBigWig.open(input_list[0])) for x in pybedtools.BedTool(input_list[1])]

In [3]:
if __name__ == '__main__':

    #parsing command line arguments
    # -----parsing command line arguments-----#
    parser = argparse.ArgumentParser(description='Training CNN model to predict STARR-seq enhancers based on chromatin accessbility and histone marks')
    parser.add_argument('-w', '--cell_types', type=str, help='comma separated string of cell_types')
    parser.add_argument('-x', '--in_dir', type=str, help='input_directory')
    parser.add_argument('-y', '--cell_name', type=str, help='name of the cell')
    parser.add_argument('-z', '--out_dir', type=str, help='output_directory')
    parser.add_argument('-a', '--track1_peaks', type=str, help='chromatin accessibility peak')
    parser.add_argument('-b', '--track2_peaks', type=str, help='ChIP-seq H3K27ac peak')
    parser.add_argument('-c', '--track3_peaks', type=str, help='ChIP-seq H3K4me3 peak')
    parser.add_argument('-d', '--track4_peaks', type=str, help='ChIP-seq H3K9ac peak')
    parser.add_argument('-e', '--track5_peaks', type=str, help='ChIP-seq H3K4me1 peak')
    parser.add_argument('-f', '--track1_bw', type=str, help='chromatin accessibility bigWig')
    parser.add_argument('-g', '--track2_bw', type=str, help='ChIP-seq H3K27ac bigWig')
    parser.add_argument('-i', '--track3_bw', type=str, help='ChIP-seq H3K4me3 bigWig')
    parser.add_argument('-j', '--track4_bw', type=str, help='ChIP-seq H3K9ac bigWig')
    parser.add_argument('-k', '--track5_bw', type=str, help='ChIP-seq H3K4me1 bigWig')

    cell_type = "H1"

    #simulate command line input
    seqdir = "/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/" + cell_type + "/"
    cmdline_str='-w ' + " HepG2,K562,A549,HCT116,MCF-7 " + \
        ' -x ' + "/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/encoded_2overlap/DNase/" + \
        ' -y ' + "H1" + \
        ' -z ' + "/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/output/" + \
        ' -a ' + seqdir+cell_type+".DNase-seq.narrowPeak" + \
        ' -b ' + seqdir+cell_type+".ChIP-seq.H3K27ac.narrowPeak" + \
        ' -c ' + seqdir+cell_type+".ChIP-seq.H3K4me3.narrowPeak" + \
        ' -d ' + seqdir+cell_type+".ChIP-seq.H3K9ac.narrowPeak" + \
        ' -e ' + seqdir+cell_type+".ChIP-seq.H3K4me1.narrowPeak" + \
        ' -f ' + seqdir+cell_type+".DNase-seq.bigWig" + \
        ' -g ' + seqdir+cell_type+".ChIP-seq.H3K27ac.bigWig" + \
        ' -i ' + seqdir+cell_type+".ChIP-seq.H3K4me3.bigWig" + \
        ' -j ' + seqdir+cell_type+".ChIP-seq.H3K9ac.bigWig" + \
        ' -k ' + seqdir+cell_type+".ChIP-seq.H3K4me1.bigWig"

    seq_names = ["DNase", "H3K27ac", "H3K4me3", "H3K9ac", "H3K4me1"]
    window_size = 4000

    #check if the files are there
    args = parser.parse_args(cmdline_str.split())
    args.cell_types = args.cell_types.split(",")
    for cell in args.cell_types:
        for seq in seq_names:
            pos_file = args.in_dir + cell + "." + seq + ".pos.tsv"
            if not os.path.exists(pos_file):
                print(pos_file + " file does not exist")
                exit(1)
            neg_file = args.in_dir + cell + "." + seq + ".neg.tsv"
            if not os.path.exists(neg_file):
                print(neg_file + " file does not exist")
                exit(1)

    for key, value in vars(args).items():
        if key == "cell_types" or key == "in_dir" or key == "out_dir" or key == "cell_name":
            continue
        else:
            if not os.path.exists(value):
                print(key + " argument file does not exist")
                exit(1)
    print("all files found!")

    #construct a set of autosome + X chromosome names
    chromosomes = []
    for i in range(1,23):
        chromosomes.append("chr"+str(i))
    chromosomes.append("chrX")
    print(chromosomes)
    print("all files found!")

all files found!
['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
all files found!


In [4]:
    #generate regions for genome wide predictions
    hg38_windows = pybedtools.BedTool().window_maker(genome="hg38", w=window_size, s=500)
    hg38_windows = hg38_windows.filter(pybedtools.featurefuncs.greater_than, window_size-1)

    #temp_chrom = chromosomes[20]
    temp_chrom = chromosomes[int(os.environ['chr_num'])]
    print(temp_chrom)

    hg38_windows = hg38_windows.filter(lambda x: x.chrom == temp_chrom).sort()

    #remove ENCODE blacklist regions
    if not os.path.exists('./hg38.blacklist.bed.gz'):
        url = 'http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg38-human/hg38.blacklist.bed.gz'
        wget.download(url, './hg38.blacklist.bed.gz')
    blacklist = pybedtools.BedTool('./hg38.blacklist.bed.gz')
    validation_regions = hg38_windows - blacklist

    validation_regions.saveas(args.out_dir + args.cell_name + "." + temp_chrom + ".validation_regions.bed")

    print(validation_regions.count())


chr9
276782


In [5]:
    #this part is painfully slow due to pyBigWig queries to continuous bigWig file
    #perhaps bedtools -> bigWigAverageOverBed is faster?
    #for the sake of clarity, the code below is kept

    p = mp.Pool(5)
    input_list = [[x, args.out_dir + args.cell_name + "." + temp_chrom + "." + "validation_regions.bed"] 
                  for x in [args.track1_bw, args.track2_bw, args.track3_bw, args.track4_bw, args.track5_bw]]
    signal_files = p.map(get_signal, input_list)
    p.close()
    p.join()

    print("finished multiprocess IO")
    sys.stdout.flush()
    
    #reformat the validation values
    valid_chromAcc = [np.array(i) for i in signal_files[0]]
    valid_chip1 = [np.array(i) for i in signal_files[1]]
    valid_chip2 = [np.array(i) for i in signal_files[2]]
    valid_chip3 = [np.array(i) for i in signal_files[3]]
    valid_chip4 = [np.array(i) for i in signal_files[4]]
    
    del signal_files
    gc.collect()

    x_validation = []
    for i in range(validation_regions.count()):
        x_validation.append(np.array([valid_chromAcc[i], valid_chip1[i], valid_chip2[i], valid_chip3[i], valid_chip4[i]]))
    x_validation = np.nan_to_num(np.array(x_validation, dtype=float))
    x_validation[:, [0, 2], :] = x_validation[:, [2, 0], :]
    print(x_validation.shape)

    x_validation = np.expand_dims(x_validation, axis=3)
    print(x_validation.shape)

    model = create_model(width=int(window_size/10))
    es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    adam = Adam(lr=5e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=9e-5)
    model.compile(loss='binary_crossentropy', optimizer=adam, 
                  metrics = ['accuracy', auroc, auprc, f1_m, recall_m, precision_m])
    model.load_weights('./saved_models/DNase_hg38.v3.h5')
        
    y_validation = model.predict(x_validation).ravel()
    print(y_validation.shape)

    #format into bed with proper regions
    df = pd.read_csv(args.out_dir + args.cell_name + "." + temp_chrom + "." + "validation_regions.bed", sep="\t",header=None)
    df[4] = "pred"
    df[5] = y_validation
    df.to_csv(args.out_dir + args.cell_name + "." + temp_chrom + "." + "prediction_regions.bed", sep="\t",header=None, index=False)
    
    #filter for positive predictions
    df[df[5]>0.5].to_csv(args.out_dir + args.cell_name + "." + temp_chrom + "." + "prediction_pos_regions.50.bed", sep="\t",header=None, index=False)
    

['/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/H1/H1.DNase-seq.bigWig', '/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/output/H1.chr9.validation_regions.bed']
['/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/H1/H1.ChIP-seq.H3K27ac.bigWig', '/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/output/H1.chr9.validation_regions.bed']
['/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/H1/H1.ChIP-seq.H3K4me3.bigWig', '/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/output/H1.chr9.validation_regions.bed']
['/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/H1/H1.ChIP-seq.H3K4me1.bigWig', '/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/output/H1.chr9.validation_regions.bed']
['/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/datasets/H1/H1

/gpfs/ysm/scratch60/gerstein/zc264/ChromVar/enhancer-prediction/encode/dev/models/v3.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  model = Model(input=[input_size], output=[pred_output])


(276782,)
